# Very naive deep learning on the vector of surrounding bases

---

### Data

Naive feature vectors. The original sequence of validation/test and train data does not overlap! ( but train data points can overlap with train data points, and test-validation can overlap with test-validation data ) This overlapping does not lead to unintentional label leakage!



### Notes


---

Instruct theano to use gpu

In [11]:
import os
os.environ['THEANO_FLAGS']='device=gpu'

import sys
sys.path.append('../my_modules')
from loading_utils import read_my_data

import subprocess
import time

import os,subprocess
workdir='/mnt/Data1/ribli/methylation_code/modelling'
subprocess.call(['mkdir',workdir])
os.chdir(workdir)

### Load data

In [2]:
_,train_x,train_y = read_my_data(
    fname='../prepare_data/big_train_feat_vect.csv')
_,valid_x,valid_y = read_my_data(
    fname='../prepare_data/big_val_feat_vect.csv')
test_id,test_x,test_y = read_my_data(
    fname='../prepare_data/big_test_feat_vect.csv')

#make it image like
train_x,valid_x,test_x=[x.reshape((-1,1,1000,1)) for x in (train_x,valid_x,test_x)]

Loading data... 
Loading data... 
Loading data... 


### Build Convnet

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D,MaxPooling2D

input_dim=train_x.shape[2]
activation='relu'
loss='binary_crossentropy'
optimizer='adadelta'
init='uniform'
pool_size=(8,1)
window_size=5
dense_n=64

model = Sequential()

#Convolution layer 1
model.add(Convolution2D(20,window_size,1, border_mode='valid',input_shape=(1,input_dim,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

#Convolution layer 2
model.add(Convolution2D(50,window_size,1, border_mode='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

#Dense layer
model.add(Flatten())
model.add(Dense(dense_n,activation=activation))
#model.add(Dropout(0.5))

#final layer
model.add(Dense(1, activation='sigmoid'))

#compile model
model.compile(loss=loss,optimizer=optimizer,class_mode='binary')

Using Theano backend.
/mnt/Data1/ribli/tools/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


### Train test model

In [4]:
def fit_keras_model(model,train_x,train_y,test_x,test_y):
    start=time.time()
    #train it
    callb_hist=model.fit(train_x,train_y,nb_epoch = 20,
                         show_accuracy=True,verbose=1)
    #predict
    train_pred=model.predict_classes(train_x).ravel()
    test_pred=model.predict_classes(test_x).ravel()

    #check errors
    print 'train score:',list((train_pred==train_y)).count(True)/float(len(train_y))
    print 'test score:',list((test_pred==test_y)).count(True)/float(len(test_y))

    print 'It took:',time.time()-start    
    return train_pred,test_pred

In [5]:
N_train,N_test=118000,10000
train_pred,test_pred=fit_keras_model(
    model,train_x[:N_train],train_y[:N_train],test_x[:N_test],test_y[:N_test])

Epoch 1/20
118000/118000 [==============================] - 45s - loss: 5.3190 - acc: 0.5525    
Epoch 2/20
118000/118000 [==============================] - 45s - loss: 0.4868 - acc: 0.7713    
Epoch 3/20
118000/118000 [==============================] - 45s - loss: 0.4380 - acc: 0.8043    
Epoch 4/20
118000/118000 [==============================] - 45s - loss: 0.4151 - acc: 0.8173    
Epoch 5/20
118000/118000 [==============================] - 45s - loss: 0.4037 - acc: 0.8253    
Epoch 6/20
118000/118000 [==============================] - 45s - loss: 0.3898 - acc: 0.8335    
Epoch 7/20
118000/118000 [==============================] - 45s - loss: 0.3780 - acc: 0.8400    
Epoch 8/20
118000/118000 [==============================] - 45s - loss: 0.3693 - acc: 0.8452    
Epoch 9/20
118000/118000 [==============================] - 45s - loss: 0.3630 - acc: 0.8475    
Epoch 10/20
118000/118000 [==============================] - 45s - loss: 0.3540 - acc: 0.8520    
Epoch 11/20
118000/118000 [==

### Save test predictions

In [10]:
test_pred=model.predict_classes(test_x).ravel()

import pandas as pd
import numpy as np
result=pd.DataFrame({'id':test_id,'label':test_y,'prediction':test_pred})
result['error']=np.abs(result['label']-result['prediction'])
result.head()

20000/20000 [==============================] - 3s     


,id,label,prediction,error
0,cg19752143,1,1,0
1,cg05219517,0,0,0
2,cg05218696,1,1,0
3,cg09329621,1,1,0
4,cg17608706,1,1,0


In [12]:
result.to_csv('cnn_test_preds.csv',sep='\t',index=False,Header=True)